In [4]:
from transformers import RobertaTokenizer

import torch

In [68]:
MAX_LENGTH = 512
classes = ['Fake', 'Real']

In [11]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
model = torch.load("model/roberta_predictor.pth", map_location=device)
model = model.to(device)

In [88]:
text_example = "The State Department told the Republican National Committee that it could not find any emails to or from Hillary Clinton\'s former IT specialist, who managed her private email server during her tenure as secretary of state before going on to work for the agency, according to a court filing made public Monday.\n\n\n\nThe government\'s revelation in U.S. District Court in Washington came in answer to a lawsuit by the Republican National Committee. The RNC had sued over its public records request for all work-related emails sent to or received by Clinton\'s former aide, Bryan Pagliano, between 2009 and 2013, the years of Clinton\'s tenure as America\'s top diplomat. The lawsuit also pressed for other State Department records from the Clinton era.\n\nThe RNC\'s filing said lawyers for the agency had informed them in discussions that 'the State Department has represented that no responsive records exist' for any Pagliano emails. Pagliano was hired at the agency after reportedly setting up Clinton\'s server in 2009, but the lack of any official State Department emails raises the question whether he limited his email traffic using a private account, much like Clinton did during her four years as secretary, or whether his government emails were deleted.\n\n\n\nA State Department official said Monday that the agency possessed emails from Pagliano from the period after Clinton\'s term had ended, when he continued to work as a technology contractor.\n\nAgency spokeswoman Elizabeth Trudeau then added Monday evening that some Pagliano emails dating from Clinton\'s tenure had been recovered from agency officials\' files and turned over to other organizations, including Senate investigators.\n\n\n\n'We have previously produced through FOIA and to Congress emails sent and received by Mr. Pagliano during Secretary Clinton\'s tenure,' Trudeau said in a public statement. At least one email, which was sent in November 2012 to Clinton from Pagliano -- but possibly from his private email address -- was released as part of 30,000 Clinton emails made public by the agency over the past year.\n\n\n\nA spokesman for the RNC said the organization stood by its description of the discussions with lawyers for the State Department. The group said in its filing that 'the State Department has represented that no responsive records exist.'\n\n\n\nRaj Shah, the RNC spokesman, added, 'It\'s hard to believe that an IT staffer who set up Hillary Clinton\'s reckless email server never sent or received a single work-related email in the four years he worked at the State Department.' Clinton\'s campaign officials declined to comment in response to questions from The Associated Press.\n\n\n\nTrudeau said the State Department is working with Congress and several public records requesters to provide relevant material. She also said agency officials continue to search for 'Mr. Pagliano\'s emails, which the department may have otherwise retained.' Trudeau also said the department would respond further to the RNC in court.\n\n\n\nState Department officials told Senate investigators last year they could not find a file containing Pagliano\'s work emails during Clinton\'s tenure, an assertion first reported by Politico.\n\n\n\nFox News reported in March that Pagliano has revealed several details about Clinton\'s personal email system to investigators, including\xa0who had access to it– as well as when and what devices were used. An intelligence source close to the case told Fox News that Pagliano has been a 'devastating witness' to Clinton.\n\nThe one email sent by Pagliano that surfaced among Clinton\'s 30,000 emails was sent to Clinton was a November 2012 birthday greeting. He wished her 'Happy Birthday Madam Secretary. To many more!'\n\n\n\nPagliano\'s email address was censored, unlike numerous official State Department addresses that are listed in Clinton\'s emails -- suggesting he may have sent the message from a private address.\n\nClinton did not reply directly to Pagliano. Instead, she sent a copy of an email to an aide with the instruction 'Pls respond.'\n\n \n\n The Associated Press contributed to this report."

In [89]:
inputs = tokenizer(text_example, padding='max_length', truncation=True, max_length=MAX_LENGTH, return_tensors="pt")
vec, mask = inputs['input_ids'].to(device), inputs['attention_mask'].to(device)

In [90]:
model.eval()

with torch.inference_mode():
    pred = model(vec, attention_mask=mask)
    
probs = torch.softmax(pred.logits, dim=1)
label_id = torch.argmax(probs, dim=1)
label = classes[label_id]

label, probs.squeeze()[label_id].item()

('Real', 0.9998480081558228)

In [1]:
from predictor import news_classifier

In [5]:
news_classifier.predict('BLA BLA BLA')

('Fake', 0.7378960251808167)

In [1]:
from db_model.database import AsyncSessionLocal, init_db
from db_model.repository import create_prediction, get_or_create_request_ip, delete_oldest_predictions

In [2]:
await init_db()

2024-03-04 23:32:35,533 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-04 23:32:35,534 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("request_ips")
2024-03-04 23:32:35,534 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-04 23:32:35,536 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("predictions")
2024-03-04 23:32:35,537 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-03-04 23:32:35,538 INFO sqlalchemy.engine.Engine COMMIT


In [4]:
await get_or_create_request_ip("qewrt") 

2024-03-04 23:33:26,370 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-04 23:33:26,371 INFO sqlalchemy.engine.Engine SELECT request_ips.id, request_ips.ip 
FROM request_ips 
WHERE request_ips.ip = ?
2024-03-04 23:33:26,372 INFO sqlalchemy.engine.Engine [cached since 23.67s ago] ('qewrt',)
2024-03-04 23:33:26,374 INFO sqlalchemy.engine.Engine INSERT INTO request_ips (ip) VALUES (?)
2024-03-04 23:33:26,374 INFO sqlalchemy.engine.Engine [cached since 23.67s ago] ('qewrt',)
2024-03-04 23:33:26,376 INFO sqlalchemy.engine.Engine COMMIT
2024-03-04 23:33:26,382 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-03-04 23:33:26,383 INFO sqlalchemy.engine.Engine SELECT request_ips.id, request_ips.ip 
FROM request_ips 
WHERE request_ips.id = ?
2024-03-04 23:33:26,384 INFO sqlalchemy.engine.Engine [cached since 23.66s ago] (3,)
2024-03-04 23:33:26,385 INFO sqlalchemy.engine.Engine ROLLBACK


3